In [1]:
from bs4 import BeautifulSoup
import pandas as pd
import requests as r

## Munging Data

In [15]:
df = pd.read_csv('ava-raw.csv')

In [16]:
df.dtypes

Unnamed: 0                           int64
Accident and Rescue Summary:        object
Aspect:                             object
Avalanche Problem:                  object
Avalanche Type:                     object
Buried - Fully:                    float64
Buried - Partly:                   float64
Carried:                           float64
Caught:                            float64
Comments:                           object
Coordinates:                        object
Depth:                              object
Elevation:                          object
Injured:                           float64
Killed:                              int64
Location Name or Route:             object
Observation Date:                   object
Observer Name:                      object
Occurence Time:                     object
Occurrence Date:                    object
Region:                             object
Slope Angle:                       float64
Snow Profile Comments:              object
Terrain Sum

In [17]:
df.describe()

,Unnamed: 0,Buried - Fully:,Buried - Partly:,Carried:,Caught:,Injured:,Killed:,Slope Angle:,Video:,killed
count,96.000000,67.000000,24.000000,75.000000,76.000000,6.000000,96.000000,42.000000,0,96.000000
mean,47.500000,1.179104,1.208333,1.653333,1.697368,1.166667,1.187500,37.857143,NaN,1.187500
std,27.856777,0.423741,0.508977,1.156572,1.189058,0.408248,0.509127,5.616055,NaN,0.509127
min,0.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,10.000000,NaN,1.000000
25%,23.750000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,36.000000,NaN,1.000000
50%,47.500000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,38.000000,NaN,1.000000
75%,71.250000,1.000000,1.000000,2.000000,2.000000,1.000000,1.000000,40.000000,NaN,1.000000
max,95.000000,3.000000,3.000000,7.000000,7.000000,2.000000,4.000000,50.000000,NaN,4.000000


In [18]:
df.columns

Index(['Unnamed: 0', 'Accident and Rescue Summary:', 'Aspect:',
       'Avalanche Problem:', 'Avalanche Type:', 'Buried - Fully:',
       'Buried - Partly:', 'Carried:', 'Caught:', 'Comments:', 'Coordinates:',
       'Depth:', 'Elevation:', 'Injured:', 'Killed:',
       'Location Name or Route:', 'Observation Date:', 'Observer Name:',
       'Occurence Time:', 'Occurrence Date:', 'Region:', 'Slope Angle:',
       'Snow Profile Comments:', 'Terrain Summary:', 'Trigger:',
       'Trigger: additional info:', 'Vertical:', 'Video:', 'Weak Layer:',
       'Weather Conditions and History:', 'Width:', 'coordinates', 'killed',
       'occurrence-date', 'region-forecaster', 'region-forecaster-1',
       'trigger', 'url'],
      dtype='object')

In [19]:
df = df.rename(columns={x:x.replace(':', '') for x in df.columns})

In [22]:
df.Aspect.value_counts()

Northeast    25
North        14
Northwest     9
East          9
West          3
Southeast     3
Southwest     1
South         1
Name: Aspect, dtype: int64

In [23]:
df['Avalanche Problem'].value_counts()

Persistent Slab    13
Storm Slab          3
Wind Slab           3
Deep Slab           3
Wet Slab            2
Glide               1
Name: Avalanche Problem, dtype: int64

In [24]:
df['Avalanche Type'].value_counts()

Hard Slab       27
Soft Slab       24
Cornice Fall     1
Wet Slab         1
Name: Avalanche Type, dtype: int64

In [25]:
df.Depth

0     2.5'
1       4'
2       4'
3      18"
4       8"
5       2'
6       3'
7       2'
8      16"
9       3'
10    2.5'
11     16"
12     NaN
13    3.5'
14      8'
15    3.5'
16      3'
17      2'
18      4'
19    4.5'
20     12"
21    4.5'
22    3.5'
23    3.5'
24     20"
25    2.5'
26      3'
27      3'
28     15"
29     12"
      ... 
66     NaN
67     NaN
68     NaN
69     NaN
70     NaN
71     NaN
72      3'
73     NaN
74      5'
75      4'
76      2'
77    3.5'
78      2'
79      2'
80      2'
81      2'
82     18"
83    3.5'
84    2.5'
85      2'
86      3'
87      4'
88      2'
89      3'
90      3'
91     NaN
92      4"
93     NaN
94      3'
95     NaN
Name: Depth, dtype: object

In [27]:
import re

def to_inches(orig):
    reg = r"""(((\d*\.)?\d*)')?(((\d*\.)?\d*)")?"""
    txt = str(orig)
    if txt == 'nan':
        return orig
    mo = re.search(reg, txt)
    feet = mo.group(2) or 0
    inches = mo.group(5) or 0
    return float(feet)*12 + float(inches)

to_inches('18"'), to_inches("2.5'")

(18.0, 30.0)

In [28]:
df['depth_inches'] = df.Depth.apply(to_inches)

In [29]:
df.depth_inches

0     30
1     48
2     48
3     18
4      8
5     24
6     36
7     24
8     16
9     36
10    30
11    16
12   NaN
13    42
14    96
15    42
16    36
17    24
18    48
19    54
20    12
21    54
22    42
23    42
24    20
25    30
26    36
27    36
28    15
29    12
      ..
66   NaN
67   NaN
68   NaN
69   NaN
70   NaN
71   NaN
72    36
73   NaN
74    60
75    48
76    24
77    42
78    24
79    24
80    24
81    24
82    18
83    42
84    30
85    24
86    36
87    48
88    24
89    36
90    36
91   NaN
92     4
93   NaN
94    36
95   NaN
Name: depth_inches, dtype: float64

In [30]:
df.depth_inches.describe()

count    61.000000
mean     32.475410
std      17.625367
min       0.000000
25%      24.000000
50%      30.000000
75%      42.000000
max      96.000000
Name: depth_inches, dtype: float64